<a href="https://colab.research.google.com/github/sachinbose-ds/Automated-Response-Generation-for-Customer-Support/blob/main/test3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from datasets import load_dataset
from transformers import BartTokenizer, BartForConditionalGeneration, Trainer, TrainingArguments
from transformers import DataCollatorForSeq2Seq

# Step 1: Load the dataset
dataset = load_dataset("Kaludi/Customer-Support-Responses")

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['query', 'response'],
        num_rows: 74
    })
})

In [ ]:
# prompt: rename featurename 'query' as question and response as 'answer'

dataset = dataset.rename_column("query", "question")
dataset = dataset.rename_column("response", "answer")


In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 74
    })
})

In [ ]:
# Step 2: Initialize the tokenizer and model
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large')

In [ ]:
# Step 3: Preprocess the dataset
def preprocess_function(examples):
    inputs = examples["question"]
    targets = examples["answer"]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=1024, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/74 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4016: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [ ]:
# Step 4: Initialize the data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)


In [ ]:
# Step 5: Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    #predict_with_generate=True,
    fp16=True,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 74
    })
})

In [ ]:
# prompt: split dataset into train and test and add it into tokenized_datasets

train_test_split = tokenized_datasets["train"].train_test_split(test_size=0.3, shuffle=False, seed=42)

tokenized_datasets["train"] = train_test_split["train"]
tokenized_datasets["test"] = train_test_split["test"]


In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 51
    })
    test: Dataset({
        features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 23
    })
})

In [ ]:
# Step 6: Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [ ]:
# Step 7: Train the model
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,1.384238
2,No log,1.099502
3,No log,1.020432


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


TrainOutput(global_step=39, training_loss=1.843948755508814, metrics={'train_runtime': 717.9481, 'train_samples_per_second': 0.213, 'train_steps_per_second': 0.054, 'total_flos': 3991366483968.0, 'train_loss': 1.843948755508814, 'epoch': 3.0})

In [ ]:
# Step 8: Save the model
model.save_pretrained("./finetuned_bart")
tokenizer.save_pretrained("./finetuned_bart")


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


('./finetuned_bart/tokenizer_config.json',
 './finetuned_bart/special_tokens_map.json',
 './finetuned_bart/vocab.json',
 './finetuned_bart/merges.txt',
 './finetuned_bart/added_tokens.json')

In [ ]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=9fb8368caa8ccda949ed9aa565298afec2d49cd806a2da9c793764bb3737d3b9
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
# Step 9: Evaluate the model
metric = load_metric("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = [[label] for label in labels]
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    return {k: round(v, 4) for k, v in result.items()}

results = trainer.evaluate()
print(results)

The repository for rouge contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/rouge.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


{'eval_loss': 1.0204322338104248, 'eval_runtime': 13.5731, 'eval_samples_per_second': 1.695, 'eval_steps_per_second': 0.442, 'epoch': 3.0}


In [51]:
# Step 10: Implementing a demo in Jupyter notebook
import ipywidgets as widgets
from IPython.display import display

# Function to generate a response
def generate_response(question):
    inputs = tokenizer(question, return_tensors="pt", max_length=1024, truncation=True)
    outputs = model.generate(inputs.input_ids, max_length=1024, num_beams=4, early_stopping=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Create widgets
input_box = widgets.Textarea(placeholder='Type your question here...', description='Question:')
output_box = widgets.Textarea(placeholder='Generated response will appear here...', description='Response:', disabled=True)
button = widgets.Button(description='Generate Response')

# Define the button click event
def on_button_clicked(b):
    question = input_box.value
    response = generate_response(question)
    output_box.value = response

button.on_click(on_button_clicked)

# Display the widgets
display(input_box, button, output_box)

Textarea(value='', description='Question:', placeholder='Type your question here...')

Button(description='Generate Response', style=ButtonStyle())

Textarea(value='', description='Response:', disabled=True, placeholder='Generated response will appear here...…